In [1]:
import networkx as nx
import functools

with open('./inputs/day21.txt', 'r') as f:
    final_codes = list(map(lambda x: x.strip(), f.readlines()))

keypad_graph = nx.DiGraph()
keypad_graph.add_edges_from([
    ('7', '4', {'direction': 'v'}), ('8', '5', {'direction': 'v'}), ('9', '6', {'direction': 'v'}),
    ('4', '1', {'direction': 'v'}), ('5', '2', {'direction': 'v'}), ('6', '3', {'direction': 'v'}),
    ('2', '0', {'direction': 'v'}), ('3', 'A', {'direction': 'v'}),
    ('0', '2', {'direction': '^'}), ('A', '3', {'direction': '^'}), ('1', '4', {'direction': '^'}),
    ('2', '5', {'direction': '^'}), ('3', '6', {'direction': '^'}), ('4', '7', {'direction': '^'}),
    ('5', '8', {'direction': '^'}), ('6', '9', {'direction': '^'}),
    ('7', '8', {'direction': '>'}), ('4', '5', {'direction': '>'}), ('1', '2', {'direction': '>'}),
    ('8', '9', {'direction': '>'}), ('5', '6', {'direction': '>'}), ('2', '3', {'direction': '>'}),
    ('0', 'A', {'direction': '>'}),
    ('9', '8', {'direction': '<'}), ('6', '5', {'direction': '<'}), ('3', '2', {'direction': '<'}),
    ('A', '0', {'direction': '<'}), ('8', '7', {'direction': '<'}), ('5', '4', {'direction': '<'}),
    ('2', '1', {'direction': '<'}),
])

arrow_graph = nx.DiGraph()
arrow_graph.add_edges_from([
    ('^', 'v', {'direction': 'v'}), ('A', '>', {'direction': 'v'}),
    ('v', '^', {'direction': '^'}), ('>', 'A', {'direction': '^'}),
    ('<', 'v', {'direction': '>'}), ('^', 'A', {'direction': '>'}), ('v', '>', {'direction': '>'}),
    ('A', '^', {'direction': '<'}), ('>', 'v', {'direction': '<'}), ('v', '<', {'direction': '<'}),
])

@functools.cache
def get_arrow_num_steps(start_button, end_button, num_inception_bots):
    if num_inception_bots == 0:
        return 1
    else:
        m = float('inf')
        for p in nx.all_shortest_paths(arrow_graph, start_button, end_button):
            path_directions = [arrow_graph.get_edge_data(p[i], p[i+1])['direction'] for i in range(len(p) - 1)] + ['A']
            path_length_idk = get_arrow_num_steps(
                start_button='A',
                end_button=path_directions[0],
                num_inception_bots=num_inception_bots-1
            )
            for i in range(len(path_directions) - 1):
                path_length_idk += get_arrow_num_steps(
                    start_button=path_directions[i],
                    end_button=path_directions[i+1],
                    num_inception_bots=num_inception_bots-1
                )

            m = min(m, path_length_idk)

        return m

def get_num_num_steps(start_num, end_num, num_inception_bots):
    m = float('inf')
    for p in nx.all_shortest_paths(keypad_graph, start_num, end_num):
        path_directions = [keypad_graph.get_edge_data(p[i], p[i+1])['direction'] for i in range(len(p) - 1)] + ['A']
        path_length_idk = get_arrow_num_steps(
            start_button='A',
            end_button=path_directions[0],
            num_inception_bots=num_inception_bots
        )
        for i in range(len(path_directions) - 1):
            path_length_idk += get_arrow_num_steps(
                start_button=path_directions[i],
                end_button=path_directions[i+1],
                num_inception_bots=num_inception_bots
            )

        m = min(m, path_length_idk)

    return m

def get_code_complexity(keypad_code, num_inception_bots):
    keypad_code = 'A' + keypad_code
    final_code_length = sum(get_num_num_steps(keypad_code[i], keypad_code[i+1], num_inception_bots) for i in range(len(keypad_code) - 1))
    return final_code_length * int(keypad_code[1:-1])

print('Answer to Day 21, Part 1:', sum(get_code_complexity(c, 2) for c in final_codes))
print('Answer to Day 21, Part 2:', sum(get_code_complexity(c, 25) for c in final_codes))

Answer to Day 21, Part 1: 270084
Answer to Day 21, Part 2: 329431019997766


In [2]:
import torch

with open('./inputs/day22.txt', 'r') as f:
    initial_numbers = list(map(int, f.readlines()))

def evolve_secret(secret_num, n=1):
    for _ in range(n):
        secret_num = (secret_num ^ (secret_num * 64)) % 16777216
        secret_num = (secret_num ^ int(secret_num / 32)) % 16777216
        secret_num = (secret_num ^ secret_num * 2048) % 16777216

    return secret_num

print('Answer to Day 22, Part 1:', sum(evolve_secret(n, 2000) for n in initial_numbers))

def get_prices(secret):
    return torch.tensor([int(str(secret)[-1:])] + [int(str(secret := evolve_secret(secret))[-1:]) for _ in range(1999)], dtype=int)

all_sequences = {}
for n in initial_numbers:
    d = {}
    prices = get_prices(n)
    diffs = prices.diff()
    for i in range(len(diffs) - 3):
        t = tuple(diffs[i: i+4].tolist())
        if not t in d:
            d[t] = int(prices[i+4])

    all_sequences[n] = d

all_keys = set()
for init_number in initial_numbers:
    all_keys = all_keys.union(all_sequences[init_number].keys())

best_score = 0
for k in all_keys:
    score = sum(all_sequences[n].get(k, 0) for n in initial_numbers)
    best_score = max(score, best_score)

print('Answer to Day 22, Part 2:', best_score)

Answer to Day 22, Part 1: 18694566361
Answer to Day 22, Part 2: 2100


In [3]:
import networkx as nx

with open('./inputs/day23.txt', 'r') as f:
    connections = list(map(lambda x: tuple(x.strip().split('-')), f.readlines()))

lan_graph = nx.Graph()
lan_graph.add_edges_from(connections)

part1_ans = sum(any(n[0] == 't' for n in c) for c in nx.simple_cycles(lan_graph, length_bound=3))
print('Answer to Day 23, Part 1:', part1_ans)

for n in lan_graph.nodes:
    s = nx.subgraph(lan_graph, [n] + list(lan_graph.neighbors(n))).copy()

    while len(s.edges) < sum(range(len(s.nodes))):
        node_to_remove = min(s.nodes, key=lambda x: len(list(s.neighbors(x))))
        s.remove_node(node_to_remove)

    if len(s.nodes) == 13:
        print('Answer to Day 23, Part 2:', ','.join(sorted(s.nodes)))
        break

Answer to Day 23, Part 1: 1423
Answer to Day 23, Part 2: gt,ha,ir,jn,jq,kb,lr,lt,nl,oj,pp,qh,vy


day 24 part 2 notes to self:
```
---
1-bit addition:

x00: ?
y00: ?

x00 XOR y00 -> z00
x00 AND y00 -> z01

---
2-bit addition:

x00: ?
x01: ?
y00: ?
y01: ?

x00 XOR y00 -> z00
x00 AND y00 -> and00

x01 XOR y01 -> xor01
x01 AND y01 -> and01
xor01 XOR and00 -> z01
xor01 AND and00 -> altcarry01

and01 OR altcarry01 -> z02

---
3-bit addition:

x00: ?
x01: ?
x02: ?
y00: ?
y01: ?
y02: ?

x00 XOR y00 -> z00
x00 AND y00 -> carry00

x01 XOR y01 -> xor01
x01 AND y01 -> and01
xor01 XOR carry00 -> z01
xor01 AND carry00 -> altcarry01
and01 OR altcarry01 -> carry01

x02 XOR y02 -> xor02
x02 AND y02 -> and02
xor02 XOR carry01 -> z02
xor02 AND carry01 -> altcarry02

and02 OR altcarry02 -> z03
```

In [4]:
from ortools.sat.python import cp_model
import networkx as nx

with open('./inputs/day24.txt', 'r') as f:
    init_vals, gates = f.read().split('\n\n')
    init_vals = {v.split(':')[0]: int(v.split(':')[1]) for v in init_vals.split('\n')}

model = cp_model.CpModel()

all_var_names = set(init_vals.keys())
for line in gates.split('\n'):
    a, _, b, _, c = line.split(' ')
    all_var_names.add(a)
    all_var_names.add(b)
    all_var_names.add(c)

all_vars = {}
for n in all_var_names:
    all_vars[n] = model.new_bool_var(n)

for v in init_vals:
    model.add(all_vars[v] == init_vals[v])

for line in gates.split('\n'):
    a, opp, b, _, c = line.split(' ')
    a, b, c = all_vars[a], all_vars[b], all_vars[c]
    if opp == 'XOR':
        model.add(~c != a + b)
        model.add(~c != a + b - 2)
    elif opp == 'AND':
        model.add(2 * c <= a + b)
        model.add(c + 1 >= a + b)
    elif opp == 'OR':
        model.add(2 * c >= a + b)
        model.add(c <= a + b)

solver = cp_model.CpSolver()
status = solver.solve(model)
if status == cp_model.OPTIMAL:
    part1_ans = int(''.join(str(solver.value(all_vars[x])) for x in sorted(filter(lambda x: x[0] == 'z', all_var_names), reverse=True)), 2)

print('Answer to Day 24, Part 1:', part1_ans)

g = nx.DiGraph()
for line in gates.split('\n'):
    a, opp, b, _, c = line.split(' ')
    g.add_edge(a, c, opp=opp)
    g.add_edge(b, c, opp=opp)

edge_opps = nx.get_edge_attributes(g, 'opp')

bad_nodes = set()
for node in g.nodes:
    preds = list(g.predecessors(node))
    sucs = list(g.successors(node))
    # XORs should lead into all but the last z
    if node[0] == 'z' and node != 'z45' and edge_opps[(preds[0], node)] != 'XOR':
        bad_nodes.add(node) # this finds 3 of 8

    if len(preds) > 0 and len(sucs) > 0 and edge_opps[(preds[0], node)] == 'AND' and edge_opps[(node, sucs[0])] != 'OR' and 'x00' not in preds:
        bad_nodes.add(node) # here's one!
        # this one actually revealed that the AND and XOR operations on x31 and y31 were mixed
        # so I'll just lazily manually add the last bad node below

for edge in g.edges:
    # XORs should either come from x/y or go towards a z
    if edge_opps[edge] == 'XOR' and not (edge[0][0] in 'xy' or edge[1][0] == 'z'):
        bad_nodes.add(edge[1]) # here's 3 more

bad_nodes.add('rvc')

print('Answer to Day 24, Part 2:', ','.join(sorted(bad_nodes)))

Answer to Day 24, Part 1: 45923082839246
Answer to Day 24, Part 2: jgb,rkf,rrs,rvc,vcg,z09,z20,z24


In [5]:
import torch
import itertools

with open('./inputs/day25.txt', 'r') as f:
    schematics = f.read().split('\n\n')

schematics = [torch.tensor([list(map(lambda x: x=='#', l)) for l in s.split()], dtype=int) for s in schematics]
locks = list(filter(lambda x: sum(x[0, :])==5, schematics))
keys = list(filter(lambda x: sum(x[0, :])==0, schematics))

part1_ans = 0
for l, k in itertools.product(locks, keys):
    if (k + l).max() < 2:
        part1_ans += 1

print('Answer to Day 25, Part 1:', part1_ans)
print('Answer to Day 25, Part 2:', '⭐️⭐️⭐️')

Answer to Day 25, Part 1: 3301
Answer to Day 25, Part 2: ⭐️⭐️⭐️
